# Predict Neutrino Direction with an LSTM

Using a Tensorflow LSTM layer using the event time steps to the input to predict the Neutrino Direction azimuth and zenith angle

## Imports

In [1]:
# Standard library imports
import os
import random
import math
import logging

# Third-party library imports
import numpy as np
import pandas as pd
import dask.dataframe as dd

# Typing imports
from typing import List, Tuple


## Variables

In [2]:
# Parameters
BATCH = 1 # What batch file to use?
EXCLUDE_AUXILIARY = True # Whether to exclude auxiliary pulses
IS_TRAINING = True # Whether to train the model
# If either the event or time limit is reached the process will exit
EVENT_LIMIT = 3000
TIME_LIMIT_HOURS = 1
PULSE_AMOUNT = 200 # Amount of pulses to use for features
FEATURES= set(('charge', 'x', 'y', 'z')) # Which features to use for training
DATA_TYPES={'A': 'int32', 'B': 'category', 'C': 'float32'}

# Directories
DATA_DIR = "data"
SET = 'train' if IS_TRAINING else 'test'

# logging
LOG_LEVEL = logging.INFO

## Logging

In [3]:
# Setup logging
logging.basicConfig(filename='artifacts/info.log', level=LOG_LEVEL, format='%(asctime)s %(levelname)s %(message)s')

## Functions

In [5]:
def seed_it_all(seed=7):
    """ Attempt to be Reproducible """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
seed_it_all(10)

### For optimization

In [8]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        logging.info(f'Optimizing col {col}')
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    logging.info('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    logging.info('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

def import_data(file: str):
    """create a dataframe and optimize its memory usage"""
    function_name = f"read_{file.split('.')[-1]}"
    function = getattr(pd, function_name)
    df = function(file)
    df = reduce_mem_usage(df)
    return df

def get_event_df(batch_df: dd.DataFrame, sensor_geometry: pd.DataFrame, event_id: int) -> pd.DataFrame:
    """
    Get a DataFrame for a specific event.

    Parameters:
    train_batch_df (pandas.DataFrame): The batch DataFrame.
    sensor_geometry (pandas.DataFrame): The sensor geometry DataFrame.
    event_id (str): The event identifier.

    Returns:
    pandas.DataFrame: A DataFrame containing data for the specified event.
    """
    if EXCLUDE_AUXILIARY:
        batch_df = batch_df[~batch_df['auxiliary']].drop(columns=['auxiliary'])
    
    event_df = batch_df[batch_df['event_id'] == event_id].compute()
        
    event_df = pd.merge(
        left=event_df,
        right=sensor_geometry,
        how='inner',
        on='sensor_id'
    ),
    
    ## Drop columns that are not needed for prediction
    return event_df.drop(columns=['event_id', 'sensor_id'])

In [ ]:
def get_input_vector(batch: dd.DataFrame, event_id: int) -> pd.DataFrame:
    """Changes the rows of a dataframe to columns

    Args:
        df (pd.DataFrame): The dataframe to be converted that currently has observations in rows

    Returns:
        pd.DataFrame: A single observation in columns
    """
    df = make_input_vector_shape(df)
    df = df.stack().reset_index()
    df['features'] = df['level_0'].astype(str) + '_' + df['level_1']
    df = df.drop(columns=['level_0','level_1']).set_index('features')
    df = df.T.set_index(pd.Index([event_id]))
    df.index.name = 'event_id'
    
    return pd.merge(
        df, 
        meta_dfd[meta_dfd['event_id']== event_id].compute()[[ 'event_id','azimuth','zenith' ]], 
        on='event_id', 
        how='inner'
    ).set_index('event_id')

## Load the dataframes

In [20]:
sensor_geometry_dfd = dd.read_csv(f'{DATA_DIR}/sensor_geometry.csv',
    blocksize=1024 ** 3 # 1GB
)
sensor_geometry_dfd.head(1)

,sensor_id,x,y,z
0,0,-256.14,-521.08,496.03


In [21]:
meta_dfd = dd.read_parquet(f'{DATA_DIR}/{SET}_meta.parquet', 
    n_partitions= 2,                                      
    blocksize=1024 ** 3 # 1GB
)
meta_dfd.head(1)

,batch_id,event_id,first_pulse_index,last_pulse_index,azimuth,zenith
0,1,24,0,60,5.029555,2.087498


In [22]:
test_batch_dfd = dd.read_parquet(f'{DATA_DIR}/{SET}/batch_1.parquet', 
     n_partitions= 2,                                      
    blocksize=1024 ** 3 # 1GB
).reset_index()


test_batch_dfd.head(1)

,event_id,sensor_id,time,charge,auxiliary
0,24,3918,5928,1.325,True


## Build the dataset

In [29]:
merged_dfd = dd.merge(test_batch_dfd, sensor_geometry_dfd, on='sensor_id', how='inner')

merged_dfd.head(3)

,event_id,sensor_id,time,charge,auxiliary,x,y,z
0,24,3918,5928,1.325,True,303.41,335.64,206.58
1,2743,3918,10813,0.975,True,303.41,335.64,206.58
2,3007,3918,10290,0.925,False,303.41,335.64,206.58


In [31]:
sorted_dfd = merged_dfd.sort_values(by=['event_id', 'time'])
sorted_dfd.head(3)

,event_id,sensor_id,time,charge,auxiliary,x,y,z
0,24,3918,5928,1.325,True,303.41,335.64,206.58
5364,24,4157,6115,1.175,True,-145.45,374.24,212.73
10172,24,3520,6492,0.925,True,505.27,257.88,-174.60


In [ ]:
merged_df.drop(columns=[])


## Build the model

In [4]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

2023-02-17 20:46:32.378313: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 20:46:33.260423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-17 20:46:33.260436: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-17 20:46:34.319346: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(64, input_shape=(PULSE_AMOUNT, len(FEATURES))))
model.add(Dense(2, activation='linear')) # set the number of output neurons to 2 and the activation function to linear

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model 
# with input sequence (num_samples, num_timesteps, num_features),
# and the azimuth and zenith angle values for each sequence.
model.fit(input_sequences, output_values, batch_size=32, epochs=100)

# Evaluate the model
loss = model.evaluate(test_sequences, test_values)

## For scoring

In [ ]:
def angular_dist_score(az_true:float, zen_true:float, az_pred:float, zen_pred:float):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    The lower the angle, the more similar the two vectors are meaning the score is better.
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two Cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against numerical instability
    # that might otherwise occur from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))